In [ ]:
import sys
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

sys.path.append(os.path.join(os.getcwd(), 'models'))

from ndlinear_net import NdLinearMNISTNet
from linear_net import LinearMNISTNet

# heck device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


ImportError: cannot import name 'NdLinearMNISTNet' from 'ndlinear_net' (c:\Users\omsri\OneDrive\Desktop\EnsembleAi\models\ndlinear_net.py)

In [ ]:
# Define a transformation for MNIST: convert grayscale to RGB and then to a tensor
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor()
])

# Download and load MNIST dataset
train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset  = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("Number of training samples:", len(train_dataset))
print("Number of test samples:", len(test_dataset))


Number of training samples: 60000
Number of test samples: 10000


In [ ]:
def train_model(model, train_loader, optimizer, criterion, epochs=10):
    """Train the given model and return loss history."""
    loss_history = []
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            # For NdLinearMNISTNet, we need to make sure we convert from (B,3,28,28) to (B,28,28,3)
            # The models take care of that inside the forward method, so we don't need additional changes here.
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        avg_loss = running_loss / len(train_loader)
        loss_history.append(avg_loss)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")
    return loss_history

def evaluate_model(model, test_loader):
    """Evaluate the model on test data and return the accuracy."""
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    accuracy = 100 * correct / total
    return accuracy


In [ ]:
# Instantiate the NdLinear model and move to device
ndlinear_model = NdLinearMNISTNet().to(device)
optimizer_ndlinear = optim.Adam(ndlinear_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

print("Training NdLinear Model...")
ndlinear_loss_history = train_model(ndlinear_model, train_loader, optimizer_ndlinear, criterion, epochs=10)
ndlinear_accuracy = evaluate_model(ndlinear_model, test_loader)
print(f"NdLinear Model Test Accuracy: {ndlinear_accuracy:.2f}%")

Training NdLinear Model...
Epoch 1/10, Loss: 0.5923
Epoch 2/10, Loss: 0.3131
Epoch 3/10, Loss: 0.3054
Epoch 4/10, Loss: 0.2978
Epoch 5/10, Loss: 0.2945
Epoch 6/10, Loss: 0.2877
Epoch 7/10, Loss: 0.2837
Epoch 8/10, Loss: 0.2797
Epoch 9/10, Loss: 0.2777
Epoch 10/10, Loss: 0.2762
NdLinear Model Test Accuracy: 92.03%


In [ ]:
# Instantiate the Linear model and move to device
linear_model = LinearMNISTNet().to(device)
optimizer_linear = optim.Adam(linear_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

print("Training Linear Model...")
linear_loss_history = train_model(linear_model, train_loader, optimizer_linear, criterion, epochs=10)
linear_accuracy = evaluate_model(linear_model, test_loader)
print(f"Linear Model Test Accuracy: {linear_accuracy:.2f}%")


Training Linear Model...
Epoch 1/10, Loss: 0.1854
Epoch 2/10, Loss: 0.0815
Epoch 3/10, Loss: 0.0549
Epoch 4/10, Loss: 0.0433
Epoch 5/10, Loss: 0.0330
Epoch 6/10, Loss: 0.0291
Epoch 7/10, Loss: 0.0245
Epoch 8/10, Loss: 0.0179
Epoch 9/10, Loss: 0.0206
Epoch 10/10, Loss: 0.0195
Linear Model Test Accuracy: 98.13%


In [ ]:
epochs = np.arange(1, 11)

plt.figure(figsize=(10, 6))
plt.plot(epochs, ndlinear_loss_history, label="NdLinear Model")
plt.plot(epochs, linear_loss_history, label="Linear Model")
plt.xlabel("Epoch")
plt.ylabel("Average Loss")
plt.title("Training Loss Comparison")
plt.legend()
plt.grid(True)
plt.show()

NameError: name 'np' is not defined

In [ ]:
print(f"Final Test Accuracy for NdLinear Model: {ndlinear_accuracy:.2f}%")
print(f"Final Test Accuracy for Linear Model: {linear_accuracy:.2f}%")


Test Accuracy: 92.18%
